# Basic settings

In [15]:
(ext:chdir "~/Development/clasp/extensions/cando/src/tests/boron/" t)

0

#### Create all paths.

In [2]:
(leap.core::clear-path)
(add-path "~/Development/clasp/extensions/cando/src/tests/boron/")
(add-path "~/Development/amber16/dat/leap/cmd/")
(add-path "~/Development/amber16/dat/leap/lib/")
(add-path "~/Development/amber16/dat/leap/parm/")
(add-path "~/Development/amber16/dat/leap/prep/")
(add-path "~/Development/amber16/dat/antechamber/")

(#P"/Users/tuj24515/Development/amber16/dat/antechamber/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/prep/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/parm/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/lib/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/cmd/"
 #P"/Users/tuj24515/Development/clasp/extensions/cando/src/tests/boron/")

In [3]:
(start-swank)

swank-loader -> /Users/tuj24515/Development/slime/swank-loader.lisp

"Read: (DEFPACKAGE :SWANK-LOADER (:USE :CL) (:EXPORT :INIT :DUMP-IMAGE :LIST-FASLS :*SOURCE-DIRECTORY* :*FASL-DIRECTORY*))
" 
"Read: (IN-PACKAGE :SWANK-LOADER)
" 
"Read: (DEFVAR *SOURCE-DIRECTORY* (MAKE-PATHNAME :NAME NIL :TYPE NIL :DEFAULTS (OR *LOAD-PATHNAME* *DEFAULT-PATHNAME-DEFAULTS*)) The directory where to look for the source.)
" 
"Read: (DEFPARAMETER *SYSDEP-FILES* (QUOTE ((SWANK CLASP) (SWANK GRAY))))
" 
"Read: (DEFPARAMETER *IMPLEMENTATION-FEATURES* (QUOTE (:ALLEGRO :LISPWORKS :SBCL :CLOZURE :CMU :CLISP :CCL :CORMAN :CORMANLISP :ARMEDBEAR :GCL :ECL :SCL :MKCL :CLASP)))
" 
"Read: (DEFPARAMETER *OS-FEATURES* (QUOTE (:MACOSX :LINUX :WINDOWS :MSWINDOWS :WIN32 :SOLARIS :DARWIN :SUNOS :HPUX :UNIX)))
" 
"Read: (DEFPARAMETER *ARCHITECTURE-FEATURES* (QUOTE (:POWERPC :PPC :X86 :X86-64 :X86_64 :AMD64 :I686 :I586 :I486 :PC386 :IAPX386 :SPARC64 :SPARC :HPPA64 :HPPA :ARM :ARMV5L :ARMV6L :ARMV7L :ARM64 :PENTIUM3 :PENTIUM4 

T

#### Load the GAFF force field and boronic ester frcmod file.

In [4]:
(clear-force-field)
(load-atom-type-rules "ATOMTYPE_GFF.DEF")

(#<FORCE-FIELD ATOMTYPE_GFF.DEF>)

In [5]:
(load-amber-params "gaff.dat")

SIMPLE-WARNING: 
Warning!  Skipping 10-12 hbond
SIMPLE-WARNING: 
Warning!  Skipping force field atom equivalences


#<FORCE-FIELD gaff.dat>

In [6]:
(load-atom-type-rules "boron.def")
(load-atom-type-rules "water_gaff.def")

(#<FORCE-FIELD water_gaff.def> #<FORCE-FIELD boron.def> #<FORCE-FIELD gaff.dat>
 #<FORCE-FIELD ATOMTYPE_GFF.DEF>)

In [7]:
(load-amber-params "frcmod.boronic-ester")
(gethash :default leap.core::*force-fields*)

(#<FORCE-FIELD frcmod.boronic-ester> #<FORCE-FIELD water_gaff.def>
 #<FORCE-FIELD boron.def> #<FORCE-FIELD gaff.dat>
 #<FORCE-FIELD ATOMTYPE_GFF.DEF>)

#### Merge force field and define parameter ff.

In [8]:
(:= *ff* (leap.core::merged-force-field))

SIMPLE-WARNING: 
Implement merge of FFVdwDb
SIMPLE-WARNING: 
Implement merge of FFVdwDb
SIMPLE-WARNING: 
Implement merge of FFVdwDb
SIMPLE-WARNING: 
Implement merge of FFVdwDb
SIMPLE-WARNING: 
Implement merge of FFVdwDb


*FF*

## Boronic ester 01

#### Load the arylboronic ester from a Chemdraw file.

In [9]:
(defparameter *b01agg* (cando:load-chem-draw-aggregate "boronic-ester01.cdxml"))

*B01AGG*

In [10]:
(progn
  (defparameter *stereocenters*
    (sort (cando:gather-stereocenters *b01agg*) #'string< :key #'chem:get-name))
  (cando:set-stereoisomer-func *stereocenters* (constantly :S) :show t)
  (let ((quat-matcher (core:make-cxx-object 'chem:chem-info)))
    (chem:compile-smarts quat-matcher "[C&H0&D4]")
    (chem:map-atoms nil (lambda (a) (when (chem:matches quat-matcher a)
                  (chem:set-configuration a :S)
                  (format t "Set atom ~a to :S~%" (chem:get-name a))))
            *b01agg*)))

C_108  S
C_110  S
C_133  S
C_135  S
C_1968 S
C_1969 S
C_1970 S
C_1981 S
C_1982 S
C_2003 S
C_2004 S
C_2005 S
C_2016 S
C_2017 S
C_208  S
C_210  S
C_233  S
C_235  S
C_722  S
C_723  S
C_724  S
C_725  S
C_726  S
Set atom C_210 to :S
Set atom C_235 to :S
Set atom C_135 to :S
Set atom C_110 to :S


NIL

#### Minimize the energy of the molecule.

In [11]:
(chem:assign-types *ff* *b01agg*)

NIL

In [12]:
(jostle *b01agg*)

NIL

In [13]:
(defparameter *b01e* (energy:minimize *b01agg* :force-field *ff* :restraints-on t))

======= Starting Steepest Descent Minimizer
---Stage-Seconds--Step-----Alpha---Dir-------------Energy-----------RMSforce
 minSDnP       0     1  0.000000   0.0              0.000         177890.970
 minSDnP       0     2  0.000445   0.0     1588511678.328          91568.653
 minSDnP       0     3  0.000347   0.0      656842495.318          48481.054
 minSDnP       0     4  0.000395   0.0      357561573.795          34977.415
 minSDnP       0     5  0.000331   0.0      228680981.153          23957.606
 minSDnP       0     6  0.000378   0.0      160327460.936          19902.319
 minSDnP       0     7  0.000344   0.0      116753282.674          15410.041
 minSDnP       0     8  0.000381   0.0       88115026.073          13434.206
 minSDnP       0     9  0.000356   0.0       67566917.701          10978.440
 minSDnP       0    10  0.000383   0.0       52882223.913           9737.516
---Stage-Seconds--Step-----Alpha---Dir-------------Energy-----------RMSforce
 minSDnP       0    11  0.000359

*B01E*

In [14]:
(nglv::show-aggregate *b01agg*)

A Jupyter Widget

In [20]:
(:= *trinag* (load-mol2 "trinag.mol2"))

*TRINAG*

In [21]:
(chem:fill-in-implicit-hydrogens *trinag*)

NIL

In [ ]:
(chem:mapatoms )

In [22]:
(nglv:show-aggregate *trinag*)

A Jupyter Widget

In [57]:
(defparameter *b02agg* (cando:load-chem-draw-aggregate "boronic-ester03.cdxml"))

*B02AGG*

In [58]:
(progn
  (defparameter *stereocenters*
    (sort (cando:gather-stereocenters *b02agg*) #'string< :key #'chem:get-name))
  (cando:set-stereoisomer-func *stereocenters* (constantly :S) :show t)
  (let ((quat-matcher (core:make-cxx-object 'chem:chem-info)))
    (chem:compile-smarts quat-matcher "[C&H0&D4]")
    (chem:map-atoms nil (lambda (a) (when (chem:matches quat-matcher a)
                  (chem:set-configuration a :S)
                  (format t "Set atom ~a to :S~%" (chem:get-name a))))
            *b02agg*)))

C_133  S
C_135  S
C_208  S
C_210  S
C_233  S
C_235  S
C_2572 S
C_2574 S
C_2582 S
C_2584 S
C_2592 S
C_2594 S
Set atom C_235 to :S
Set atom C_2594 to :S
Set atom C_2584 to :S
Set atom C_210 to :S
Set atom C_135 to :S
Set atom C_2574 to :S


NIL

In [59]:
(chem:assign-types *ff* *b02agg*)

NIL

In [60]:
(jostle *b02agg*)

NIL

In [61]:
(defparameter *b02e* (energy:minimize *b02agg* :force-field *ff* :restraints-on t))

======= Starting Steepest Descent Minimizer
---Stage-Seconds--Step-----Alpha---Dir-------------Energy-----------RMSforce
 minSDnP       0     1  0.000000   0.0              0.000         280703.967
 minSDnP       0     2  0.000395   0.0     4444556839.280         124690.644
 minSDnP       0     3  0.000435   0.0     1909315983.297          81542.629
 minSDnP       0     4  0.000360   0.0     1010561487.465          51303.693
 minSDnP       0     5  0.000404   0.0      612771511.802          40433.258
 minSDnP       0     6  0.000343   0.0      402485461.536          29045.062
 minSDnP       0     7  0.000389   0.0      279869205.972          24852.406
 minSDnP       0     8  0.000340   0.0      201550492.540          19004.914
 minSDnP       0     9  0.000391   0.0      148761977.430          16886.164
 minSDnP       0    10  0.000344   0.0      112226618.096          13255.911
---Stage-Seconds--Step-----Alpha---Dir-------------Energy-----------RMSforce
 minSDnP       0    11  0.000395

SIMPLE-WARNING: 
Estimated angle term :ne-:c2-:ne  T0 -> 109.500000 degrees K -> 70.000000 kCal/rad^2
SIMPLE-WARNING: 
Estimated angle term :c2-:c2-:ne  T0 -> 109.500000 degrees K -> 70.000000 kCal/rad^2


*B02E*

In [62]:
(nglv::show-aggregate *b02agg*)

A Jupyter Widget

#### Save the topology file and coordinate file.

In [44]:
(chem:set-property *b01agg* :bounding-box '(30.0 30.0 30.0))

NIL

In [45]:
(chem:assign-types *ff* *b01agg*)

NIL

In [46]:
(leap.topology:save-amber-parm-format *b01agg* "b01.top" "b01.crd" *ff* :assign-types nil)

atom1: #<ATOM :C_312/:C> atom2: #<ATOM :C_310/:C> i1: 0 i2: 3
 atom1: #<ATOM :C_312/:C> atom2: #<ATOM :H_312/:H> i1: 0 i2: 588
 atom1: #<ATOM :C_310/:C> atom2: #<ATOM :C_308/:C> i1: 3 i2: 9
 atom1: #<ATOM :C_310/:C> atom2: #<ATOM :B/:B> i1: 3 i2: 12
 atom1: #<ATOM :C_314/:C> atom2: #<ATOM :C_312/:C> i1: 6 i2: 0
 atom1: #<ATOM :C_314/:C> atom2: #<ATOM :C_298/:C> i1: 6 i2: 15
 atom1: #<ATOM :C_314/:C> atom2: #<ATOM :H_314/:H> i1: 6 i2: 585
 atom1: #<ATOM :C_308/:C> atom2: #<ATOM :C_306/:C> i1: 9 i2: 18
 atom1: #<ATOM :C_308/:C> atom2: #<ATOM :H_308/:H> i1: 9 i2: 582
 atom1: #<ATOM :B/:B> atom2: #<ATOM :OH/:O> i1: 12 i2: 24
 atom1: #<ATOM :C_298/:C> atom2: #<ATOM :C_281/:C> i1: 15 i2: 30
 atom1: #<ATOM :C_306/:C> atom2: #<ATOM :C_298/:C> i1: 18 i2: 15
 atom1: #<ATOM :C_306/:C> atom2: #<ATOM :H_306/:H> i1: 18 i2: 579
 atom1: #<ATOM :OH/:O> atom2: #<ATOM :B/:B> i1: 21 i2: 12
 atom1: #<ATOM :OH/:O> atom2: #<ATOM :HH/:H> i1: 21 i2: 639
 atom1: #<ATOM :OH/:O> atom2: #<ATOM :HH/:H> i1: 24 i2: 6

0

#### Calculate each energy parameters.

In [47]:
(chem:get-energy (chem:get-stretch-component *b01e*))

82.36554103619612d0

In [48]:
(chem:get-energy (chem:get-angle-component *b01e*))

105.9430062847246d0

In [49]:
(chem:get-energy (chem:get-dihedral-component *b01e*))

189.36024506338896d0

In [50]:
(chem:get-energy (chem:get-nonbond-component *b01e*))

240.12762920122927d0

#### Create Tip3 water molecules box.

In [51]:
(load-off "tip3pfbox.off")

(:TIP3PFBOX)

In [52]:
(nglv::show-aggregate ~tip3pfbox)

A Jupyter Widget

In [53]:
(print ~tip3pfbox)


AGGREGATE 

AGGREGATE

In [54]:
(defparameter *solvent* ~tip3pfbox)

*SOLVENT*

In [55]:
(chem:assign-types *ff* *solvent*)

NIL

#### Add Tip3 water molecules to the boronic ester.

In [56]:
(:= *b01wagg* (leap.solvate::tool-solvate-and-shell *b01agg* *solvent* '(30.0 30.0 30.0) :shell t :farness 7.0))

Total bounding box for atom centers:  81.73844693175073d0 85.77759373786844d0 66.99863433109397d0


*B01WAGG*

In [57]:
(chem:map-atoms 'list (lambda (a) (chem:get-type a)) *b01wagg*)

(:|c2| :|c2| :|c2| :|c2| :|b| :|c2| :|c2| :|oh| :|oh| :|os| :|c3| :|c3| :|n|
 :|c3| :|h1| :|c3| :|c3| :|c| :|c3| :|h1| :|oh| :|c3| :|h1| :|oh| :|c3| :|c3|
 :|c| :|c3| :|o| :|os| :|h1| :|c3| :|h2| :|os| :|c3| :|n3| :|o| :|n| :|c3|
 :|oh| :|c3| :|c| :|c3| :|c3| :|c3| :|c3| :|h1| :|o| :|oh| :|n| :|c| :|os|
 :|h1| :|c3| :|os| :|h1| :|c3| :|c| :|c3| :|o| :|n| :|b| :|c3| :|h1| :|oh|
 :|oh| :|c3| :|o| :|c2| :|c3| :|c2| :|oh| :|oh| :|h2| :|os| :|c3| :|c2| :|c2|
 :|c3| :|c2| :|c2| :|c3| :|n| :|c| :|c3| :|c3| :|h1| :|c| :|c3| :|o| :|n| :|os|
 :|h1| :|c3| :|os| :|h1| :|c3| :|c3| :|o| :|c2| :|c3| :|b| :|c3| :|h1| :|oh|
 :|oh| :|c3| :|c2| :|c2| :|c3| :|c2| :|oh| :|oh| :|h2| :|os| :|c2| :|c2| :|n|
 :|c| :|b| :|c3| :|c| :|n| :|o| :|c2| :|oh| :|oh| :|c2| :|o| :|c3| :|c2| :|c2|
 :|c2| :|c2| :|c2| :|c2| :|c2| :|c2| :|c2| :|c2| :|ha| :|ha| :|ha| :|ha| :|ha|
 :|ha| :|ha| :|ha| :|ha| :|h1| :|h1| :|h1| :|h1| :|ha| :|ha| :|ha| :|ha| :|hc|
 :|hc| :|h1| :|h1| :|h1| :|h1| :|h1| :|h1| :|h1| :|ha| :|ha| :|ha| :|h

#### Minimize the energy of the water and boronic ester.

In [58]:
(defparameter *b01we* (energy:minimize *b01wagg* :force-field *ff* :restraints-on t))

======= Starting Steepest Descent Minimizer
---Stage-Seconds--Step-----Alpha---Dir-------------Energy-----------RMSforce
 minSDnP       0     1  0.000000   0.0              0.000           7248.333
DONE absolute force test:
forceRmsMag(137.808063).LT.forceTolerance(5000.000000)
======= Starting Conjugate Gradient Minimizer
 minCGnP       0     2  0.000000   0.0              0.000            137.808
 minCGnP       0     3  0.000025  35.9           8188.222             99.704
 minCGnP       0     4  0.000127  70.9           5681.766            244.675
 minCGnP       0     5  0.000034  25.1           2823.308             50.961
 minCGnP       0     6  0.000392  12.4           1633.479             20.881
 minCGnP       0     7  0.000103  61.6           1531.649             38.159
 minCGnP       0     8  0.000037  65.4           1434.335             52.674
 minCGnP       0     9  0.000070  56.1           1175.590             13.503
 minCGnP       0    10  0.000310  70.8           1069.034  

*B01WE*

In [59]:
(nglv::show-aggregate *b01wagg*)

A Jupyter Widget

#### Save the topology file and coordinate file.

In [60]:
(chem:assign-types *ff* *b01wagg*)

NIL

In [61]:
(leap.topology:save-amber-parm-format *b01wagg* "b01_water.top" "b01_water.crd" *ff* :assign-types nil)

atom1: #<ATOM :C_312/:C> atom2: #<ATOM :C_310/:C> i1: 0 i2: 3
 atom1: #<ATOM :C_312/:C> atom2: #<ATOM :C_314/:C> i1: 0 i2: 6
 atom1: #<ATOM :C_308/:C> atom2: #<ATOM :C_310/:C> i1: 9 i2: 3
 atom1: #<ATOM :B/:B> atom2: #<ATOM :C_310/:C> i1: 12 i2: 3
 atom1: #<ATOM :C_298/:C> atom2: #<ATOM :C_314/:C> i1: 15 i2: 6
 atom1: #<ATOM :C_306/:C> atom2: #<ATOM :C_308/:C> i1: 18 i2: 9
 atom1: #<ATOM :C_306/:C> atom2: #<ATOM :C_298/:C> i1: 18 i2: 15
 atom1: #<ATOM :OH/:O> atom2: #<ATOM :B/:B> i1: 21 i2: 12
 atom1: #<ATOM :OH/:O> atom2: #<ATOM :B/:B> i1: 24 i2: 12
 atom1: #<ATOM :O/:O> atom2: #<ATOM :B/:B> i1: 27 i2: 12
 atom1: #<ATOM :C_281/:C> atom2: #<ATOM :C_298/:C> i1: 30 i2: 15
 atom1: #<ATOM :C_724/:C> atom2: #<ATOM :O/:O> i1: 33 i2: 27
 atom1: #<ATOM :C_724/:C> atom2: #<ATOM :H/:H> i1: 33 i2: 42
 atom1: #<ATOM :C_724/:C> atom2: #<ATOM :C_723/:C> i1: 33 i2: 45
 atom1: #<ATOM :N/:N> atom2: #<ATOM :C_110/:C> i1: 36 i2: 48
 atom1: #<ATOM :N/:N> atom2: #<ATOM :C_281/:C> i1: 36 i2: 30
 atom1: #<AT

0

In [62]:
(chem:get-energy (chem:get-stretch-component *b01we*))

83.20676890860902d0

In [63]:
(chem:get-energy (chem:get-angle-component *b01we*))

109.30096341569147d0

In [64]:
(chem:get-energy (chem:get-dihedral-component *b01we*))

188.36619959633745d0

In [65]:
(chem:get-energy (chem:get-nonbond-component *b01we*))

-214.9212268510093d0

In [14]:
(chem:map-residues nil (lambda (r) (print r)) ~tip3pfbox)


#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WAT> 
#<RESIDUE :WA

NIL

In [47]:
(chem:map-atoms nil (lambda (r) (print r)) *solvent*)


#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O> 
#<ATOM :H1/:H> 
#<ATOM :H2/:H> 
#<ATOM :O/:O

NIL

In [29]:
(defparameter *cat* (cando:load-chem-draw-aggregate "spiroligomer transglycosylase copy.cdxml"))

SIMPLE-ERROR: 
Could not find file spiroligomer transglycosylase copy.cdxml


NIL